## Nhúng

Trong ví dụ trước, chúng ta đã làm việc với các vector túi từ có kích thước cao với độ dài `vocab_size`, và chúng ta đã chuyển đổi rõ ràng các vector biểu diễn vị trí có kích thước thấp thành biểu diễn thưa thớt dạng one-hot. Biểu diễn one-hot này không tiết kiệm bộ nhớ. Ngoài ra, mỗi từ được xử lý một cách độc lập với nhau, vì vậy các vector mã hóa one-hot không thể biểu thị được sự tương đồng ngữ nghĩa giữa các từ.

Trong bài học này, chúng ta sẽ tiếp tục khám phá tập dữ liệu **News AG**. Để bắt đầu, hãy tải dữ liệu và lấy một số định nghĩa từ bài học trước.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Embedding là gì?

Ý tưởng của **embedding** là biểu diễn các từ bằng các vector dày đặc có kích thước thấp hơn, phản ánh ý nghĩa ngữ nghĩa của từ. Chúng ta sẽ thảo luận sau về cách xây dựng các embedding từ có ý nghĩa, nhưng hiện tại hãy chỉ nghĩ về embedding như một cách để giảm kích thước của vector từ.

Vì vậy, một lớp embedding nhận một từ làm đầu vào và tạo ra một vector đầu ra với kích thước `embedding_size` được chỉ định. Theo một cách nào đó, nó rất giống với lớp `Dense`, nhưng thay vì nhận một vector mã hóa one-hot làm đầu vào, nó có thể nhận một số đại diện cho từ.

Bằng cách sử dụng lớp embedding làm lớp đầu tiên trong mạng của chúng ta, chúng ta có thể chuyển từ mô hình bag-of-words sang mô hình **embedding bag**, nơi chúng ta đầu tiên chuyển đổi mỗi từ trong văn bản thành embedding tương ứng, sau đó tính toán một số hàm tổng hợp trên tất cả các embedding đó, chẳng hạn như `sum`, `average` hoặc `max`.

![Hình ảnh minh họa một bộ phân loại embedding cho năm từ trong chuỗi.](../../../../../translated_images/vi/embedding-classifier-example.b77f021a7ee67eee.webp)

Mạng neural phân loại của chúng ta bao gồm các lớp sau:

* Lớp `TextVectorization`, nhận một chuỗi ký tự làm đầu vào và tạo ra một tensor chứa các số đại diện cho token. Chúng ta sẽ chỉ định một kích thước từ vựng hợp lý `vocab_size` và bỏ qua các từ ít được sử dụng. Hình dạng đầu vào sẽ là 1, và hình dạng đầu ra sẽ là $n$, vì chúng ta sẽ nhận được $n$ token làm kết quả, mỗi token chứa các số từ 0 đến `vocab_size`.
* Lớp `Embedding`, nhận $n$ số và giảm mỗi số thành một vector dày đặc với độ dài được chỉ định (100 trong ví dụ của chúng ta). Do đó, tensor đầu vào có hình dạng $n$ sẽ được chuyển đổi thành tensor $n\times 100$.
* Lớp tổng hợp, tính trung bình của tensor này dọc theo trục đầu tiên, tức là nó sẽ tính trung bình của tất cả $n$ tensor đầu vào tương ứng với các từ khác nhau. Để triển khai lớp này, chúng ta sẽ sử dụng lớp `Lambda` và truyền vào nó hàm để tính trung bình. Đầu ra sẽ có hình dạng 100, và nó sẽ là biểu diễn số của toàn bộ chuỗi đầu vào.
* Bộ phân loại tuyến tính `Dense` cuối cùng.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Trong phần `summary`, trong cột **output shape**, chiều đầu tiên của tensor `None` tương ứng với kích thước minibatch, và chiều thứ hai tương ứng với độ dài của chuỗi token. Tất cả các chuỗi token trong minibatch có độ dài khác nhau. Chúng ta sẽ thảo luận cách xử lý vấn đề này trong phần tiếp theo.

Bây giờ, hãy huấn luyện mạng lưới:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Lưu ý** rằng chúng tôi đang xây dựng bộ vector hóa dựa trên một tập con của dữ liệu. Điều này được thực hiện để tăng tốc quá trình, và nó có thể dẫn đến tình huống khi không phải tất cả các token từ văn bản của chúng tôi đều có mặt trong từ vựng. Trong trường hợp này, các token đó sẽ bị bỏ qua, điều này có thể dẫn đến độ chính xác thấp hơn một chút. Tuy nhiên, trong thực tế, một tập con của văn bản thường cung cấp một ước tính từ vựng tốt.


### Xử lý kích thước chuỗi biến đổi

Hãy cùng tìm hiểu cách quá trình huấn luyện diễn ra trong các minibatch. Trong ví dụ trên, tensor đầu vào có kích thước 1, và chúng ta sử dụng các minibatch dài 128, do đó kích thước thực tế của tensor là $128 \times 1$. Tuy nhiên, số lượng token trong mỗi câu lại khác nhau. Nếu chúng ta áp dụng lớp `TextVectorization` cho một đầu vào duy nhất, số lượng token trả về sẽ khác nhau, tùy thuộc vào cách văn bản được token hóa:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Tuy nhiên, khi chúng ta áp dụng vectorizer cho nhiều chuỗi, nó phải tạo ra một tensor có hình dạng chữ nhật, vì vậy nó điền các phần tử không sử dụng bằng token PAD (trong trường hợp của chúng ta là số không):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Ở đây chúng ta có thể thấy các embeddings:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Lưu ý**: Để giảm thiểu lượng đệm, trong một số trường hợp, việc sắp xếp tất cả các chuỗi trong tập dữ liệu theo thứ tự tăng dần độ dài (hoặc, chính xác hơn, số lượng token) là hợp lý. Điều này sẽ đảm bảo rằng mỗi minibatch chứa các chuỗi có độ dài tương tự nhau.


## Nhúng ngữ nghĩa: Word2Vec

Trong ví dụ trước, lớp nhúng đã học cách ánh xạ các từ thành các biểu diễn vector, tuy nhiên, những biểu diễn này không mang ý nghĩa ngữ nghĩa. Sẽ rất hữu ích nếu chúng ta có thể học một biểu diễn vector sao cho các từ tương tự hoặc từ đồng nghĩa tương ứng với các vector gần nhau theo một khoảng cách vector nào đó (ví dụ như khoảng cách Euclide).

Để làm được điều này, chúng ta cần tiền huấn luyện mô hình nhúng trên một tập hợp văn bản lớn bằng cách sử dụng một kỹ thuật như [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Phương pháp này dựa trên hai kiến trúc chính được sử dụng để tạo ra biểu diễn phân tán của từ:

 - **Continuous bag-of-words** (CBoW), trong đó chúng ta huấn luyện mô hình để dự đoán một từ dựa trên ngữ cảnh xung quanh. Với ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, mục tiêu của mô hình là dự đoán $W_0$ từ $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** thì ngược lại với CBoW. Mô hình sử dụng cửa sổ ngữ cảnh xung quanh để dự đoán từ hiện tại.

CBoW nhanh hơn, trong khi skip-gram chậm hơn nhưng lại làm tốt hơn trong việc biểu diễn các từ ít xuất hiện.

![Hình minh họa cả hai thuật toán CBoW và Skip-Gram để chuyển đổi từ thành vector.](../../../../../translated_images/vi/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Để thử nghiệm với nhúng Word2Vec được tiền huấn luyện trên tập dữ liệu Google News, chúng ta có thể sử dụng thư viện **gensim**. Dưới đây là cách tìm các từ giống nhất với 'neural'.

> **Lưu ý:** Khi bạn tạo vector từ lần đầu tiên, việc tải xuống chúng có thể mất một khoảng thời gian!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Chúng ta cũng có thể trích xuất vector nhúng từ từ, để sử dụng trong việc huấn luyện mô hình phân loại. Vector nhúng có 300 thành phần, nhưng ở đây chúng ta chỉ hiển thị 20 thành phần đầu tiên của vector để rõ ràng:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Điều tuyệt vời về các biểu diễn ngữ nghĩa là bạn có thể thao tác mã hóa vector dựa trên ngữ nghĩa. Ví dụ, chúng ta có thể yêu cầu tìm một từ có biểu diễn vector gần nhất có thể với các từ *vua* và *phụ nữ*, và xa nhất có thể với từ *đàn ông*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Một ví dụ trên sử dụng một số phép thuật nội bộ của GenSym, nhưng logic cơ bản thực sự khá đơn giản. Một điều thú vị về các embedding là bạn có thể thực hiện các phép toán vector thông thường trên các vector embedding, và điều đó sẽ phản ánh các phép toán trên **ý nghĩa** của từ. Ví dụ trên có thể được biểu diễn dưới dạng các phép toán vector: chúng ta tính toán vector tương ứng với **KING-MAN+WOMAN** (các phép toán `+` và `-` được thực hiện trên các biểu diễn vector của các từ tương ứng), và sau đó tìm từ gần nhất trong từ điển với vector đó:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: Chúng tôi đã phải thêm một số hệ số nhỏ vào các vector *man* và *woman* - hãy thử loại bỏ chúng để xem điều gì xảy ra.

Để tìm vector gần nhất, chúng tôi sử dụng công cụ TensorFlow để tính toán một vector khoảng cách giữa vector của chúng tôi và tất cả các vector trong từ vựng, sau đó tìm chỉ số của từ có giá trị nhỏ nhất bằng cách sử dụng `argmin`.


Mặc dù Word2Vec có vẻ là một cách tuyệt vời để biểu thị ngữ nghĩa của từ, nó có nhiều hạn chế, bao gồm những điểm sau:

* Cả hai mô hình CBoW và skip-gram đều là **embedding dự đoán**, và chúng chỉ xem xét ngữ cảnh cục bộ. Word2Vec không tận dụng được ngữ cảnh toàn cục.
* Word2Vec không xem xét đến **hình thái học** của từ, tức là ý nghĩa của từ có thể phụ thuộc vào các phần khác nhau của từ, chẳng hạn như gốc từ.

**FastText** cố gắng khắc phục hạn chế thứ hai và phát triển dựa trên Word2Vec bằng cách học các biểu diễn vector cho mỗi từ và các n-gram ký tự xuất hiện trong từng từ. Các giá trị của các biểu diễn này sau đó được trung bình thành một vector tại mỗi bước huấn luyện. Mặc dù điều này thêm nhiều tính toán bổ sung vào quá trình tiền huấn luyện, nó cho phép embedding từ mã hóa thông tin về các phần của từ.

Một phương pháp khác, **GloVe**, sử dụng cách tiếp cận khác đối với embedding từ, dựa trên việc phân tích ma trận ngữ cảnh của từ. Đầu tiên, nó xây dựng một ma trận lớn đếm số lần xuất hiện của từ trong các ngữ cảnh khác nhau, sau đó cố gắng biểu diễn ma trận này trong các chiều thấp hơn theo cách giảm thiểu mất mát tái tạo.

Thư viện gensim hỗ trợ các embedding từ này, và bạn có thể thử nghiệm với chúng bằng cách thay đổi mã tải mô hình ở trên.


## Sử dụng các embedding đã được huấn luyện trước trong Keras

Chúng ta có thể sửa đổi ví dụ trên để điền trước ma trận trong lớp embedding của mình bằng các embedding ngữ nghĩa, chẳng hạn như Word2Vec. Từ vựng của embedding đã được huấn luyện trước và tập văn bản có thể không khớp nhau, vì vậy chúng ta cần chọn một trong hai. Ở đây, chúng ta sẽ khám phá hai tùy chọn khả thi: sử dụng từ vựng của tokenizer và sử dụng từ vựng từ embedding Word2Vec.

### Sử dụng từ vựng của tokenizer

Khi sử dụng từ vựng của tokenizer, một số từ trong từ vựng sẽ có embedding Word2Vec tương ứng, và một số sẽ bị thiếu. Với kích thước từ vựng là `vocab_size`, và độ dài vector embedding Word2Vec là `embed_size`, lớp embedding sẽ được biểu diễn bằng một ma trận trọng số có hình dạng `vocab_size`$\times$`embed_size`. Chúng ta sẽ điền ma trận này bằng cách duyệt qua từ vựng:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Đối với các từ không có trong từ vựng của Word2Vec, chúng ta có thể để chúng dưới dạng các số không, hoặc tạo một vector ngẫu nhiên.

Bây giờ chúng ta có thể định nghĩa một lớp nhúng với các trọng số đã được huấn luyện trước:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Lưu ý**: Lưu ý rằng chúng ta đặt `trainable=False` khi tạo `Embedding`, điều này có nghĩa là chúng ta không huấn luyện lại lớp Embedding. Điều này có thể làm giảm độ chính xác một chút, nhưng nó giúp tăng tốc quá trình huấn luyện.

### Sử dụng từ vựng embedding

Một vấn đề với cách tiếp cận trước đó là từ vựng được sử dụng trong TextVectorization và Embedding không giống nhau. Để giải quyết vấn đề này, chúng ta có thể sử dụng một trong các giải pháp sau:
* Huấn luyện lại mô hình Word2Vec trên từ vựng của chúng ta.
* Tải tập dữ liệu của chúng ta với từ vựng từ mô hình Word2Vec đã được huấn luyện trước. Từ vựng được sử dụng để tải tập dữ liệu có thể được chỉ định trong quá trình tải.

Cách tiếp cận thứ hai có vẻ dễ dàng hơn, vì vậy hãy triển khai nó. Trước tiên, chúng ta sẽ tạo một lớp `TextVectorization` với từ vựng được chỉ định, lấy từ các embeddings của Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Thư viện word embeddings của gensim chứa một hàm tiện lợi, `get_keras_embeddings`, sẽ tự động tạo lớp embeddings tương ứng của Keras cho bạn.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Một trong những lý do chúng ta không thấy độ chính xác cao hơn là vì một số từ từ tập dữ liệu của chúng ta bị thiếu trong từ vựng GloVe đã được huấn luyện trước, và do đó chúng về cơ bản bị bỏ qua. Để khắc phục điều này, chúng ta có thể huấn luyện các embedding của riêng mình dựa trên tập dữ liệu của chúng ta.


## Biểu diễn ngữ cảnh

Một hạn chế chính của các biểu diễn nhúng được huấn luyện trước truyền thống như Word2Vec là, mặc dù chúng có thể nắm bắt một phần ý nghĩa của một từ, nhưng chúng không thể phân biệt giữa các nghĩa khác nhau. Điều này có thể gây ra vấn đề cho các mô hình xử lý sau.

Ví dụ, từ 'play' có các nghĩa khác nhau trong hai câu sau:
- Tôi đã đi xem một **vở kịch** ở nhà hát.
- John muốn **chơi** với bạn bè của mình.

Các nhúng được huấn luyện trước mà chúng ta đã thảo luận biểu diễn cả hai nghĩa của từ 'play' trong cùng một nhúng. Để khắc phục hạn chế này, chúng ta cần xây dựng các nhúng dựa trên **mô hình ngôn ngữ**, được huấn luyện trên một tập hợp văn bản lớn và *hiểu* cách các từ có thể được kết hợp trong các ngữ cảnh khác nhau. Việc thảo luận về các nhúng ngữ cảnh nằm ngoài phạm vi của hướng dẫn này, nhưng chúng ta sẽ quay lại chủ đề này khi nói về các mô hình ngôn ngữ trong bài học tiếp theo.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
